## TorchServe Continuous Batching Serve Llama-2-70B on Inferentia-2
This notebook demonstrates TorchServe stream with microbatching serving Llama-2-70b on Inferentia-2 `inf2.48xlarge` with Neuron DLAMI Deep Learning AMI Neuron (Ubuntu 22.04) 20240401 and Neuron DLC [public.ecr.aws/neuron/pytorch-inference-neuronx:2.1.2-neuronx-py310-sdk2.18.0-ubuntu20.04](https://github.com/aws-neuron/deep-learning-containers?tab=readme-ov-file#pytorch-inference-neuronx)

### Installation

In [ ]:
# Activate Transformers NeuronX (PyTorch 2.1) Python venv
!source /opt/aws_neuronx_venv_transformers_neuronx/bin/activate

# Install torch-model-archiver
!pip install torch-model-archiver

# Clone Torchserve git repository
!git clone https://github.com/pytorch/serve.git

# Install dependencies, now all commands run under serve dir
!cd serve

### Create model artifacts

In [ ]:
# login in Hugginface hub
!pip install --upgrade huggingface_hub
!huggingface-cli login --token $HUGGINGFACE_TOKEN
!python examples/large_models/utils/Download_model.py --model_path model --model_name meta-llama/Llama-2-70b-hf --use_auth_token True

# Create TorchServe model artifacts
!torch-model-archiver --model-name llama-2-70b --version 1.0 --handler ts/torch_handler/distributed/base_neuronx_microbatching_handler.py -r examples/large_models/inferentia2/llama2/streamer/requirements.txt --config-file examples/large_models/inferentia2/llama2/streamer/model-config.yaml --archive-format no-archive

!mkdir model_store
!mv llama-2-70b model_store
!mv model model_store/llama-2-70b

In [ ]:
### Start docker

In [ ]:
!docker run -it -v /home/ubuntu/serve/model_store/:/home/model-server/model_store -v /home/ubuntu/serve/:/home/model-server/serve --device /dev/neuron0:/dev/neuron0  --device /dev/neuron1:/dev/neuron1  --device /dev/neuron2:/dev/neuron2  --device /dev/neuron3:/dev/neuron3  --device /dev/neuron4:/dev/neuron4  --device /dev/neuron5:/dev/neuron5 --device /dev/neuron0:/dev/neuron6 --device /dev/neuron1:/dev/neuron7  --device /dev/neuron2:/dev/neuron8  --device /dev/neuron3:/dev/neuron9  --device /dev/neuron4:/dev/neuron10  --device /dev/neuron5:/dev/neuron11 -p 8080:8080 -p 8081:8081 -p 8082:8082 -p 7070:7070 -p 7071:7071  public.ecr.aws/neuron/pytorch-inference-neuronx:2.1.2-neuronx-py310-sdk2.18.0-ubuntu20.04 bash

In [ ]:
### Inside container

In [ ]:
!pip install torchserve==0.10.0
!cd /home/model-server/serve

### Start TorchServe

In [1]:
!torchserve --ncs --start --model-store model_store --models llama-2-70b --ts-config examples/large_models/inferentia2/llama2/config.properties

SyntaxError: invalid syntax (1315009035.py, line 1)

### Run inference

In [ ]:
# Run single inference request
!python examples/large_models/utils/test_llm_streaming_response.py -m llama-2-70b -o 50 -t 2 -n 4 --prompt-text "Today the weather is really nice and I am planning on "